### TODO

get unii code from gsrs ref
get duns numbers
need understand company and fdf better-- are they the same?

In [2]:
import os
print(os.environ)

environ({'PATH': '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'HOSTNAME': '73e874bf5a39', 'JUPYTERHUB_API_TOKEN': 'eae6ab5a500549f0bfb703a6537f57a9', 'JPY_API_TOKEN': 'eae6ab5a500549f0bfb703a6537f57a9', 'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-zhengrongma', 'JUPYTERHUB_HOST': '', 'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/zhengrongma/oauth_callback', 'JUPYTERHUB_USER': 'zhengrongma', 'JUPYTERHUB_SERVER_NAME': '', 'JUPYTERHUB_API_URL': 'http://d6f381c37041:8081/hub/api', 'JUPYTERHUB_ACTIVITY_URL': 'http://d6f381c37041:8081/hub/api/users/zhengrongma/activity', 'JUPYTERHUB_BASE_URL': '/', 'JUPYTERHUB_SERVICE_PREFIX': '/user/zhengrongma/', 'MEM_LIMIT': '21474836480', 'MEM_GUARANTEE': '5368709120', 'CPU_LIMIT': '6.0', 'CPU_GUARANTEE': '1.0', 'USER': 'zhengrongma', 'NB_USER': 'zhengrongma', 'USER_ID': '1010', 'NB_UID': '1010', 'HOME': '/home/zhengrongma', 'LC_ALL': 'C.UTF-8', 'LANG': 'C.UTF-8', 'ANSIBLE_ROLES_PATH': '/ansible/roles', 'KERNEL_LAUNCH_TIMEOUT': '40', 'JPY_PARENT

In [3]:
#get list of pharmacircle files
from os import listdir
from os.path import isfile, join
path =  "/home/zhengrongma/supply_chain_project/data/pharmacircle/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

#rdd = sc.textFile("file:///supply_chain_project/data/pharmacircle/api_manufacturer.csv")
#api_manufacturer = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/api_manufacturer.csv")

In [5]:
#get number of rows in each table
def get_row_ct(path):
    row_ct = []
    for f in files:
        to_open = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv(path + f)
        row_ct.append((f, to_open.count()))
    return row_ct   
        
row_ct = get_row_ct("/home/zhengrongma/supply_chain_project/data/pharmacircle/")

In [6]:
row_ct

[('pc_form.csv', 224),
 ('pc_territory.csv', 279),
 ('api_manufacturer.csv', 110387),
 ('Sales_Company.csv', 54682),
 ('pc_molecule.csv', 3984),
 ('Sales.csv', 15971),
 ('pc_dosing_strength_compound.csv', 145463),
 ('pc_branch.csv', 1841),
 ('pc_dosing_strength_ingredient.csv', 399027),
 ('fdf_manufacturer.csv', 77561),
 ('Sales_Compound.csv', 26350),
 ('Branch_Hierarchy.csv', 1825),
 ('pc_dosing_strength.csv', 53826),
 ('pc_compound.csv', 107064),
 ('pc_compound_combined.csv', 43901),
 ('Sales_Territory.csv', 54402),
 ('pc_excipient.csv', 3385),
 ('pc_company.csv', 3547)]

In [64]:
import pyspark.sql.functions as F
for f in files:
    to_open = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv(path + f)
    print(f)
    print(to_open.agg(*(100.0*(F.count(F.col(c))/to_open.count()).alias(c) for c in to_open.columns)))

pc_form.csv
+---------------------------------------------+-------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------+
|((count(form_id) / 224) AS `form_id` * 100.0)|((count(form_name) / 224) AS `form_name` * 100.0)|((count(parent_form_id) / 224) AS `parent_form_id` * 100.0)|((count(parent_route) / 224) AS `parent_route` * 100.0)|
+---------------------------------------------+-------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------+
|                                        100.0|                                            100.0|                                                        0.0|                                                  100.0|
+---------------------------------------------+-------------------------------------------------+-----------------------------------

In [ ]:


colname = list(to_open.select('mvv').toPandas()['mvv']
)

In [8]:
import pyspark.sql.functions as F

for f in files:
    to_open = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv(path + f)
    colname = list(to_open.schema.names)
    if 'source' in colname:
        print(colname)
    if colname[0] == 'api_manufacturer':
        to_open.groupBy('source').agg(F.countDistinct(F.col('api'))).show(20, False)  
    if colname[0] == 'fdf_manufacturer':
        to_open.groupBy('source').agg(F.countDistinct(F.col('fdf'))).show(20, False)     
        

['api_manufacturer', 'source', 'duns_number', 'fei', 'api_manufacturer_address', 'country', 'api', 'unii_code', 'CAS_registry']
+-------------------+-------------------+
|source             |count(DISTINCT api)|
+-------------------+-------------------+
|China Regulatory   |2645               |
|FDA DMF            |11970              |
|CEP/COS Certificate|2410               |
|New Zealand Medsafe|2522               |
|China Api          |827                |
|India DMF          |1559               |
|FDA US Label       |1432               |
|Company Website    |19418              |
+-------------------+-------------------+

['fdf_manufacturer', 'source', 'duns_number', 'fei', 'fdf_manufacturer_address', 'country', 'product_name', 'NDC_code', 'fdf', 'applicant', 'marketing_category_name', 'dosage_form', 'route_of_administration', 'excipient_name', 'excipient_synonym']
+-------------------+-------------------+
|source             |count(DISTINCT fdf)|
+-------------------+--------------

In [9]:
# Import PySpark Library
from pyspark.sql import SparkSession
# Initialize a Spark Session
spark = SparkSession.builder.getOrCreate()
# Edit the default Config for the "eagerEval" of the Spark Session 
# Reason: Improve the formatting of table printout
# Assumption: These options are on by default, and are only meant to help optimized certain processes in Spark
# Note: Only works for 'last line' outputs, but not for print out commands 
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [10]:
api_manufacturer = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/api_manufacturer.csv")

In [11]:
api_manufacturer

api_manufacturer,source,duns_number,fei,api_manufacturer_address,country,api,unii_code,CAS_registry
Merck KGaA DE 642...,CEP/COS Certificate,null,null,null,Germany,Ascorbic acid,null,null
DR. REDDY\'S LABO...,CEP/COS Certificate,null,null,null,India,Diltiazem hydroch...,null,null
Janssen Pharmaceu...,CEP/COS Certificate,null,null,null,Ireland,Ketoconazole,null,null
KDL Biotech Limit...,CEP/COS Certificate,null,null,null,India,Amoxicillin trihy...,null,null
KDL Biotech Limit...,CEP/COS Certificate,null,null,null,India,Cloxacillin sodium,null,null
KW Pfannenschmidt...,CEP/COS Certificate,null,null,null,Germany,Theophylline,null,null
Teva Group - Bulk...,CEP/COS Certificate,null,null,null,Israel,Atenolol,null,null
Amino GmbH DE 383...,CEP/COS Certificate,null,null,null,Germany,Isoleucine,null,null
Amifarma S.L. ES ...,CEP/COS Certificate,null,null,null,Spain,Cefazolin sodium ...,null,null
Klinge Pharma & ...,CEP/COS Certificate,null,null,null,Ireland,Diclofenac sodium,null,null


In [12]:
api_manufacturer.agg(*(100.0*(F.count(F.col(c))/api_manufacturer.count()).alias(c) for c in api_manufacturer.columns))

((count(api_manufacturer) / 110387) AS `api_manufacturer` * 100.0),((count(source) / 110387) AS `source` * 100.0),((count(duns_number) / 110387) AS `duns_number` * 100.0),((count(fei) / 110387) AS `fei` * 100.0),((count(api_manufacturer_address) / 110387) AS `api_manufacturer_address` * 100.0),((count(country) / 110387) AS `country` * 100.0),((count(api) / 110387) AS `api` * 100.0),((count(unii_code) / 110387) AS `unii_code` * 100.0),((count(CAS_registry) / 110387) AS `CAS_registry` * 100.0)
99.97825830940238,99.97825830940238,2.3481025845434695,2.176886771087175,73.20517814597733,92.26811128121972,99.97191698297807,2.332702220370152,1.7656064572821075


In [13]:
# some address has no country info
null_country = api_manufacturer.filter(api_manufacturer.country.isNull())
null_country

api_manufacturer,source,duns_number,fei,api_manufacturer_address,country,api,unii_code,CAS_registry
M/s.RA Chem Pharm...,India DMF,null,null,null,null,Doxazosin Mesylate,null,null
M/s. Indoco Remed...,India DMF,null,null,null,null,Olopatadine HCl,null,null
M/s. Indoco Remed...,India DMF,null,null,null,null,Bromfenac Sodium,null,null
M/s. Parabolic Dr...,India DMF,null,null,null,null,Cefixime Trihydrate,null,null
M/s. Parabolic Dr...,India DMF,null,null,null,null,Cefpodoxime Proxetil,null,null
M/s. Parabolic Dr...,India DMF,null,null,null,null,Cefuroxime Axetil,null,null
M/s. Parabolic Dr...,India DMF,null,null,null,null,Cefdinir,null,null
M/s. Parabolic Dr...,India DMF,null,null,null,null,Cefprozil,null,null
M/s. Vitalife Lab...,India DMF,null,null,null,null,Ropinirole Hydroc...,null,null
M/s. Sharp Global...,India DMF,null,null,null,null,Menthol,null,null


In [14]:
# There are mixed upper and lower case in name
# Not every API has UNII code or Cas Number
# Not every address is availanble 

not_null_unii = api_manufacturer.filter(api_manufacturer.unii_code.isNotNull())
not_null_unii

api_manufacturer,source,duns_number,fei,api_manufacturer_address,country,api,unii_code,CAS_registry
HOMEODEL INC,FDA US Label,259068237,3008740602,null,null,ANTIMONY TRISULFIDE,F79059A38U,null
Apotheca Company,FDA US Label,844330915,1931484,null,null,ADENOSINE TRIPHOS...,5L51B4DR1G,987-65-5
Beecham Pharmaceu...,FDA US Label,595132580,3003469490,null,null,AMOXICILLIN,804826J2HU,26787-78-0
LABORATORIOS JAYO...,FDA US Label,979312485,null,null,null,WATER,059QF0KO0R,7732-18-5
Apotheca Company,FDA US Label,844330915,1931484,null,null,CONVALLARIA MAJALIS,QHH4HVF5QE,null
Apotheca Company,FDA US Label,844330915,1931484,null,null,BETULA PENDULA BARK,40S83Y133C,null
Apotheca Company,FDA US Label,844330915,1931484,null,null,PYRROLE,86S1ZD6L2C,null
"Olon, SpA",FDA US Label,543029958,3002806373,null,null,doxorubicin hydro...,82F2G7BL4E,25316-40-9
Aurobindo Pharma ...,FDA US Label,918917662,3004086884,null,null,clopidogrel bisul...,08I79HTP27,120202-66-6
Janssen Pharmaceu...,FDA US Label,080236951,1033845,null,null,tapentadol hydroc...,71204KII53,175591-09-0


In [15]:
def display_colvalue(dataset, colname):
    dataset.select(colname).distinct().show(20, False)

In [16]:
china_reg = api_manufacturer.filter(api_manufacturer.source == 'China Regulatory')
china_reg

api_manufacturer,source,duns_number,fei,api_manufacturer_address,country,api,unii_code,CAS_registry
Evonik Rexim(Nann...,China Regulatory,null,null,"10 Wenjiang Road,...",China,α-ketoleucine cal...,null,null
Hebei JiuPai Phar...,China Regulatory,null,null,"Sanxia road, econ...",China,α-Ketophenylalani...,null,null
Hebei JiuPai Phar...,China Regulatory,null,null,"Sanxia road, econ...",China,α-Ketoleucine Cal...,null,null
CHANGZHOU YABANG...,China Regulatory,null,null,LIANGCHANG EAST R...,China,α-Asarone,null,null
Cisen Pharmaceut...,China Regulatory,null,null,Tongji Tech Ind...,China,Ｃycloclenbuterol ...,null,null
Zhejiang Hisun Ph...,China Regulatory,null,null,"No.46,Waisha Road...",China,zolmitriptan,null,null
Chongqing Lummy p...,China Regulatory,null,null,"No. 2, Hua’nan 4t...",China,voriconazole,null,null
Jiangsu Wuzhong P...,China Regulatory,null,null,No.561 SixFeng Ro...,China,vitamin D3,null,null
Guangdong Xianqia...,China Regulatory,null,null,"Gongye Road,Congh...",China,tiopronin,null,null
Tangshan City Rui...,China Regulatory,null,null,Tangshan City Yut...,China,terbinafine hydro...,null,null


In [17]:
china_reg.select('api_manufacturer_address').show(20, False)  

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|api_manufacturer_address                                                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10 Wenjiang Road,Wuming County,Nanning，Guangxi                                                                                                                                       |
|Sanxia road, economy teconology area of shijiazhuang                                                                                                                                  |
|Sanxia road, economy teconology area of shijiazhuang                       

In [18]:
company_web = api_manufacturer.filter(api_manufacturer.source == 'Company Website')
company_web.select('api_manufacturer','api_manufacturer_address').show(20, False) 

+--------------------------+----------------------------------------------------------------------------------+
|api_manufacturer          |api_manufacturer_address                                                          |
+--------------------------+----------------------------------------------------------------------------------+
|APOTEX PHARMACHEM INC     |11, 34, 50 Spalding Drive||Brantford, Ontario, Canada N3T 6B8                     |
|APOTEX PHARMACHEM INC     |null                                                                              |
|APOTEX PHARMACHEM INC     |null                                                                              |
|APOTEX PHARMACHEM INC     |null                                                                              |
|APOTEX PHARMACHEM INC     |null                                                                              |
|APICORE LLC               |49 Napoleon Ct. Somerset NJ 08520 United States                             

In [19]:
FDADMF = api_manufacturer.filter(api_manufacturer.source == 'FDA DMF')
FDADMF

api_manufacturer,source,duns_number,fei,api_manufacturer_address,country,api,unii_code,CAS_registry
SPECGX LLC,FDA DMF,null,null,null,United States of ...,PARZONE BITARTRATE,null,null
FIS FABBRICA ITAL...,FDA DMF,null,null,Via Massimo D'Ant...,Italy,NITROFURANTOIN & ...,null,null
FIS FABBRICA ITAL...,FDA DMF,null,null,Via Massimo D'Ant...,Italy,CHLORDIAZEPOXIDE HCL,null,null
CAMBREX PROFARMAC...,FDA DMF,null,null,VIA EUGENIO CURIE...,Italy,CHLORDIAZEPOXIDE ...,null,null
CAMBREX PROFARMAC...,FDA DMF,null,null,"Via Curiel, 34 Pa...",Italy,HYDROCHLOROTHIAZIDE,null,null
AJINOMOTO CO INC,FDA DMF,null,null,450 Morodomichomo...,United States of ...,AMINO ACID PRODUCTS,null,null
SIEGFRIED USA LLC,FDA DMF,null,null,33 Industrial Par...,United States of ...,BUTALBITAL,null,null
FMC LITHIUM SUB F...,FDA DMF,null,null,null,null,LITHIUM CARBONATE,null,null
UNION CARBIDE COR...,FDA DMF,null,null,null,France,CARBOWAX POLYETHY...,null,null
PHARMACIA AND UPJ...,FDA DMF,null,null,null,Spain,HYDROCORTISONE MI...,null,null


In [20]:
api_pc_nonspl =  api_manufacturer.filter(api_manufacturer.source != 'FDA US Label')


### unii and cas need to be mapped and added

In [21]:
fdf_manufacturer = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/fdf_manufacturer.csv")

In [22]:
fdf_manufacturer

fdf_manufacturer,source,duns_number,fei,fdf_manufacturer_address,country,product_name,NDC_code,fdf,applicant,marketing_category_name,dosage_form,route_of_administration,excipient_name,excipient_synonym
perrigo uk limited,New Zealand Medsafe,null,null,Perrigo UK Limite...,null,milk of magnesia,null,Magnesium hydroxide,null,null,Tablet,Oral,Magnesium stearat...,null
merck sharp dohm...,New Zealand Medsafe,null,null,Merck Sharp Dohm...,null,maxalt,null,Rizatriptan benzoate,null,null,Film coated tablet,Oral,Iron oxide red|La...,null
catalent uk swind...,New Zealand Medsafe,null,null,Catalent UK Swind...,null,maxalt,null,Rizatriptan benzoate,null,null,Wafer,null,Aspartame|Gelatin...,null
catalent uk swind...,New Zealand Medsafe,null,null,Catalent UK Swind...,null,maxalt melt,null,Rizatriptan benzoate,null,null,Wafer,null,Aspartame|Gelatin...,null
astrazeneca uk ltd,New Zealand Medsafe,null,null,AstraZeneca UK Lt...,null,fluothane anaesth...,null,Halothane,null,null,Volatile liquid f...,Inhalation,null,null
merck sharp dohm...,New Zealand Medsafe,null,null,Merck Sharp Dohm...,null,aldomet,null,Methyldopate hydr...,null,null,Solution for inje...,Injection,null,null
rhonepoulenc sant...,New Zealand Medsafe,null,null,RhonePoulenc Sant...,null,rovamycin,null,Spiramycin,null,null,Tablet,Oral,null,null
pfizer australia ...,New Zealand Medsafe,null,null,Pfizer Australia ...,null,codral pain relief,null,Codeine phosphate...,null,null,Tablet,Oral,Magnesium stearat...,null
pfizer australia ...,New Zealand Medsafe,null,null,Pfizer Australia ...,null,codral original c...,null,Codeine phosphate...,null,null,Tablet,Oral,Ethanol|Hyprolose...,null
pfizer australia ...,New Zealand Medsafe,null,null,Pfizer Australia ...,null,codral cough syrup,null,Dextromethorphan ...,null,null,Syrup,Oral,Ethanol as 95% v/...,null


In [170]:
#fdf_manufacturer.filter(F.col('source')== 'New Zealand Medsafe').select('fdf').orderBy(F.length('fdf').desc()).show(20, False)
#fdf_manufacturer.filter((F.col('source')== 'New Zealand Medsafe') & (F.length('fdf') == 30)).select('fdf').orderBy(F.length('fdf').desc()).show(20, False)
fdf_manufacturer.filter((F.col('source')== 'New Zealand Medsafe') & (F.col('fdf').rlike('Betamethasone'))).select('fdf').orderBy(F.length('fdf').desc()).show(20, False)
#Betamethasone sodium phosphate

+---+
|fdf|
+---+
+---+



In [23]:
display_colvalue(fdf_manufacturer, 'source')

+-------------------+
|source             |
+-------------------+
|New Zealand Medsafe|
|Japan (PMDA)       |
|FDA US Label       |
+-------------------+



In [24]:
fdf_manufacturer.agg(*(100.0*(F.count(F.col(c))/fdf_manufacturer.count()).alias(c) for c in fdf_manufacturer.columns))

((count(fdf_manufacturer) / 77561) AS `fdf_manufacturer` * 100.0),((count(source) / 77561) AS `source` * 100.0),((count(duns_number) / 77561) AS `duns_number` * 100.0),((count(fei) / 77561) AS `fei` * 100.0),((count(fdf_manufacturer_address) / 77561) AS `fdf_manufacturer_address` * 100.0),((count(country) / 77561) AS `country` * 100.0),((count(product_name) / 77561) AS `product_name` * 100.0),((count(NDC_code) / 77561) AS `NDC_code` * 100.0),((count(fdf) / 77561) AS `fdf` * 100.0),((count(applicant) / 77561) AS `applicant` * 100.0),((count(marketing_category_name) / 77561) AS `marketing_category_name` * 100.0),((count(dosage_form) / 77561) AS `dosage_form` * 100.0),((count(route_of_administration) / 77561) AS `route_of_administration` * 100.0),((count(excipient_name) / 77561) AS `excipient_name` * 100.0),((count(excipient_synonym) / 77561) AS `excipient_synonym` * 100.0)
100.0,100.0,61.698534057064755,52.71334820334962,38.300176635164576,18.62662936269517,99.99742138445869,61.698534057064755,100.0,61.69982336483542,61.698534057064755,88.54965768878688,79.36720774616109,78.66066708784054,0.7439305836696277


In [25]:
fdf_manufacturer.filter(fdf_manufacturer.source == 'Japan (PMDA)').show(20, False)

+---------------------------------------+------------+-----------+----+-----------------------------------------------------------------+-------+--------------------------------------+--------+----------------------------------------------------------------------------------+---------+-----------------------+-------------------------------------------------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|fdf_manufacturer                       |source      |duns_number|fei |fdf_manufacturer_address                                         |country|product_name                          |NDC_code|fdf                                                                               |applicant|marketing_category_name|dosage_form                                            |route_of_administration|excipient_name                        

### pc_branch is for relationship of many different kinds, from
branch_id 6 is API or biological sup

In [26]:
pc_branch = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_branch.csv")

In [27]:
pc_branch

branch_id,abbreviation,branch_name,branch_synonym,full_name,explanation,hyperlink,top_page,type,hide_in,dds_type
1,null,DRUG DELIVERY,null,null,null,null,static_top/drug_d...,drug delivery,null,null
2,null,PHARMA & BIOTECHN...,null,null,null,null,static_top/pharma...,pharma/biotechnology,null,formulation
3,null,Pharma Equipment,null,null,null,null,null,links,null,formulation
4,null,Pharmaceutical Se...,null,null,null,null,null,links,technology/pharmabio,formulation
5,null,Pharmaceutical Se...,null,null,null,null,null,links,null,formulation
6,null,API/Biologics Sup...,Active Pharmaceut...,null,null,null,null,links,null,formulation
7,null,Pharmaceutical Ex...,null,null,null,null,null,links,null,formulation
9,null,ORAL,null,null,null,null,null,drug delivery,null,route
10,null,INJECTION,PARENTERAL,null,null,null,null,drug delivery,null,route
11,null,SKIN,CUTANEOUS,null,null,null,null,drug delivery,null,route


### Branch Hierarchy is a parent child relationship lookup table

In [28]:
bh = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/Branch_Hierarchy.csv")

In [29]:
## get unique count of parent id and child id

### number of api

In [67]:
import pyspark.sql.functions as F
api_manufacturer.agg(F.countDistinct(F.lower(F.col('api'))))

count(DISTINCT lower(api))
32500


In [68]:
api_pc_nonspl.agg(F.countDistinct(F.col('api')))

count(DISTINCT api)
36754


In [32]:
#read newport from csv file
newport = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ",").csv("/home/zhengrongma/supply_chain_project/data/newport/USP_API_Manufacturers__March2020Delivery_v2.0_wFEI.csv")

In [33]:
newport

GROUP NAME,GROUP CITY,GROUP COUNTRY,GROUP TYPE,NEWPORT CORPORATE API RATING,COMPANY NAME,FEI NUMBER,ADDR1,ADDR2,POSTALCODE,CITY,STATE,COUNTRY,ACTIVE INGREDIENT,MANUFACTURING STATUS,ACTIVE US DMF,AVAILABLE FOR REF US DMF,JAPANESE DMF,KOREAN DMF,ACTIVE COS,FDA INSPECTION DATE,GDUFA FEE PAYMENT DATE,FACILITY REGISTRATION DATE,FDA WARNING LETTER DATE
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,ferrous sulfate,Unconfirmed API a...,1,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,citicoline sodium,Commercially Avai...,0,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,metronidazole ben...,Commercially Avai...,0,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,thionordiazepam,Not Manufacturing,0,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,nordazepam,Not Manufacturing,0,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Acharya Chemicals,3010957801,"W-41, MIDC, Indl....",Ambarnath District,421501,Thane,Maharashtra,India,desipramine hydro...,Not Manufacturing,0,0,0,0,0,6/24/16 0:00,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Herbert Brown Pha...,#N/A,"W-258, M.I.D.C. P...",Dombivli (East),421 203,Thane,Maharashtra,India,clomipramine hydr...,Commercially Avai...,0,0,0,0,0,null,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Herbert Brown Pha...,#N/A,"W-258, M.I.D.C. P...",Dombivli (East),421 203,Thane,Maharashtra,India,reserpine,Not Manufacturing,0,0,0,0,0,null,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Herbert Brown Pha...,#N/A,"W-258, M.I.D.C. P...",Dombivli (East),421 203,Thane,Maharashtra,India,diazepam,Not Manufacturing,0,0,0,0,0,null,null,null,null
Acharya Chemicals,Mumbai,India,API,Potential Future,Herbert Brown Pha...,#N/A,"W-258, M.I.D.C. P...",Dombivli (East),421 203,Thane,Maharashtra,India,halquinols,Not Manufacturing,0,0,0,0,0,null,null,null,null


In [34]:
newport.agg(f.countDistinct(f.col('ACTIVE INGREDIENT')))

count(DISTINCT ACTIVE INGREDIENT)
6660


In [35]:
#lst_pc_api = api_manufacturer.select('api').collect()
lst_pc_api = list(api_manufacturer.select('api').toPandas()['api'] )
lst_np_api = list(newport.select('ACTIVE INGREDIENT').toPandas()['ACTIVE INGREDIENT'] )

In [36]:
lst_common = [item for item in lst_pc_api if item in lst_np_api ]

In [37]:
print(len(lst_common))

1603


In [38]:
lst_name = newport.schema.names

In [39]:
lst_name

['GROUP NAME',
 'GROUP CITY',
 'GROUP COUNTRY',
 'GROUP TYPE',
 'NEWPORT CORPORATE API RATING',
 'COMPANY NAME',
 'FEI NUMBER',
 'ADDR1',
 'ADDR2',
 'POSTALCODE',
 'CITY',
 'STATE',
 'COUNTRY',
 'ACTIVE INGREDIENT',
 'MANUFACTURING STATUS',
 'ACTIVE US DMF',
 'AVAILABLE FOR REF US DMF',
 'JAPANESE DMF',
 'KOREAN DMF',
 'ACTIVE COS',
 'FDA INSPECTION DATE',
 'GDUFA FEE PAYMENT DATE',
 'FACILITY REGISTRATION DATE',
 'FDA WARNING LETTER DATE']

In [40]:
lst_FDADMF = list(set(FDADMF.select('api_manufacturer').toPandas()['api_manufacturer']))

In [41]:
print(len(lst_FDADMF))

4695


In [42]:
lst_comweb = list(set(company_web.select('api_manufacturer').toPandas()['api_manufacturer']))

In [43]:
lst_common2 = [item for item in lst_FDADMF if item in lst_comweb ]

In [44]:
print(len(lst_common2))

6


In [45]:
lst_comweb[:20]

['Pharmagen Ltd.',
 'Aarambh Life Science',
 'Suzhou NeuPharma Co., Ltd.',
 'Bio Sidus S.A.',
 'Saraca Laboratories Ltd.',
 'Concord Biotech Limited',
 'Olainfarm AS',
 'BYS Chem CO., Ltd',
 'Miat S.p.A.',
 'Suzhou ChiralPharm Technology Co., Ltd.',
 'Nanjing King-friend Biochemical Pharmaceutical Co., Ltd',
 'LGM Pharma',
 'Formosa Laboratories, Inc.',
 'AARTI Healthcare',
 'Covex S.A.',
 'Lupin Ltd',
 'Recordati',
 'Aceto Fine Chem GmbH',
 'Kamada Ltd.',
 'Sichuan Credit Chemwerth Pharmaceutical Co., Ltd.']

### pc_company

In [46]:
company = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_company.csv")

In [47]:
company

company_id,name,synonym,address,city,US_state,country,company_website,telephone,year_founded,employee_number,moment_updated
3086,Crawford Healthca...,Crawford Pharmace...,King Edward Court...,Cheshire,null,United Kingdom,https://www.crawf...,+44 (0) 1565 654920,1996,null,2020-07-20 17:01:47
1021,Kyowa Kirin Inter...,ProStrakan Group Plc,Galabank Business...,Galashiels,null,United Kingdom,http://www.intern...,+44 (0) 1896 664000,1995,null,2020-06-30 09:45:31
7891,Cantabria Labs,Industrial Farmac...,"Calle Arequipa, 1...",Madrid,null,Spain,https://www.canta...,+34 677 53 07 86,1949,840,2020-04-20 17:23:17
19963,Cantabria Labs Di...,Difa Cooper S.p.A...,"Via Milano, 160 2...",Caronno Pertusella,null,Italy,https://www.canta...,(+39) 02.9659031,1961,null,2020-07-05 20:46:24
3063,Wockhardt Ltd.,null,Bandra Kurla Comp...,Mumbai,null,India,http://www.wockha...,+91 22 2653 4444,1967,6254,2020-06-17 23:31:30
5759,Hyundai Pharmaceu...,null,"204-4, Nonhyun-do...",Seoul,null,South Korea,http://www.hyunda...,+82-2-2600-3951,1965,null,2020-06-29 15:13:33
7,Abeona Therapeuti...,Access Pharmaceut...,1330 Avenue of th...,New York,NY-New York,United States of ...,http://abeonather...,646-813-4712,1974,83,2020-06-27 13:22:20
392,AMAG Pharmaceutic...,Advanced Magnetic...,1100 Winter Stree...,Waltham,MA-Massachusetts,United States of ...,https://www.amagp...,617.498.3300,1981,467,2020-06-15 11:05:21
2282,RHEI Pharmaceutic...,RHEI Pharmaceutic...,"Room 2004, 20/F C...",Hong Kong,null,Hong Kong,null,+1 289 234 2586,2004,null,2020-06-13 11:22:56
2340,Norgine B.V.,null,"Cross Towers, Ant...",Amsterdam,null,Netherlands,http://www.norgin...,+31 (0)20 567 09 00,1906,null,2020-07-10 13:54:55


In [48]:
import pyspark.sql.functions as F
company.agg(*(100.0*(F.count(F.col(c))/company.count()).alias(c) for c in company.columns))

((count(company_id) / 3547) AS `company_id` * 100.0),((count(name) / 3547) AS `name` * 100.0),((count(synonym) / 3547) AS `synonym` * 100.0),((count(address) / 3547) AS `address` * 100.0),((count(city) / 3547) AS `city` * 100.0),((count(US_state) / 3547) AS `US_state` * 100.0),((count(country) / 3547) AS `country` * 100.0),((count(company_website) / 3547) AS `company_website` * 100.0),((count(telephone) / 3547) AS `telephone` * 100.0),((count(year_founded) / 3547) AS `year_founded` * 100.0),((count(employee_number) / 3547) AS `employee_number` * 100.0),((count(moment_updated) / 3547) AS `moment_updated` * 100.0)
100.0,100.0,50.09867493656611,99.49252889766,99.57710741471666,30.10995207217367,99.88722864392444,80.57513391598535,92.1623907527488,91.51395545531436,14.942204680011278,100.0


In [49]:
pc_branch.agg(*(100.0*(F.count(F.col(c))/pc_branch.count()).alias(c) for c in pc_branch.columns))

((count(branch_id) / 1841) AS `branch_id` * 100.0),((count(abbreviation) / 1841) AS `abbreviation` * 100.0),((count(branch_name) / 1841) AS `branch_name` * 100.0),((count(branch_synonym) / 1841) AS `branch_synonym` * 100.0),((count(full_name) / 1841) AS `full_name` * 100.0),((count(explanation) / 1841) AS `explanation` * 100.0),((count(hyperlink) / 1841) AS `hyperlink` * 100.0),((count(top_page) / 1841) AS `top_page` * 100.0),((count(type) / 1841) AS `type` * 100.0),((count(hide_in) / 1841) AS `hide_in` * 100.0),((count(dds_type) / 1841) AS `dds_type` * 100.0)
100.0,11.298207495926126,99.94568169473112,36.93644758283541,0.27159152634437805,4.942965779467681,21.944595328625745,0.16295491580662683,99.89136338946226,1.1950027159152634,98.91363389462249


In [50]:
#pc_branch.select('type').distinct().show()
pc_branch.select('branch_name').distinct().show(10, False)

+---------------------------+
|branch_name                |
+---------------------------+
|Poration, Iontophoresis    |
|Cancer, Testicular         |
|Pulmonary Embolism         |
|Ocular Absorption Enhancers|
|Cancer, CCL                |
|Photodynamic, Topical      |
|Intrauterine Devices       |
|Conventional Melt Tablets  |
|Infections                 |
|Hypoparathyroidism         |
+---------------------------+
only showing top 10 rows



### pc_compound

In [51]:
compound = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_compound.csv")

In [52]:
#contain lot of partnership relation
compound

compound_id,product_name,product_synonym,molecule_id_1,molecule_id_2,molecule_id_3,molecule_id_4,type,generic_or_innovator,therapeutic_category_id,therapeutic_group_id,phase,indication_explanation,route_branch_id,drug_delivery_branch_id_1,drug_delivery_branch_id_2,drug_delivery_branch_id_3,drug_delivery_branch_id_4,product_originator_company_id_1,product_originator_company_id_2,dds_technology_owner_company_id_1,dds_technology_owner_company_id_2,discovery_technology_owner_company_id,previous_partner_company_id_1,previous_partner_company_id_2,compound_company_id_1,compound_company_id_2,compound_company_id_3,compound_company_id_4,company_role_1,company_role_2,company_role_3,company_role_4,company_explanation,territory_id,manufacturing_details,API_manufacturing_partner_company_id_1,explanation_API_1,API_manufacturing_partner_company_id_2,explanation_API_2,API_manufacturing_partner_company_id_3,explanation_API_3,formulation_manufacturing_partner_company_id_1,explanation_formulation_manufacturing_1,formulation_manufacturing_partner_company_id_2,explanation_formulation_manufacturing_2,formulation_manufacturing_partner_company_id_3,explanation_formulation_manufacturing_3,parent_compound_id,approval_date,market_launch_date,registration_date,activity,last_activity_change,last_changed_fields,group_id,moment_updated,moment_inserted,highest_phase
2,Darvocet A500 Tab...,null,2,887,null,null,pharma/bio,generic,1454,1,Marketed,null,9,1382,null,null,null,5,null,null,null,null,null,null,1465,null,null,null,product owner,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2003-09-10,null,null,not active,2012-02-06,"Company,Marketing...",2,2019-03-26 08:42:56,2006-03-28 15:18:45,Marketed
5192,Balacet 325 Tablets,null,2,887,null,null,pharma/bio,generic,1454,6898,Marketed,Indicated for the...,9,1382,null,null,null,1429,null,null,null,null,null,null,999,1056,2507,null,product owner,co-promotion,co-promotion,null,"In April 2007, Co...",6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2004-05-07,null,null,not active,2017-04-19,"Clinical Trial,Dr...",5192,2017-10-04 15:05:27,2006-04-07 13:15:53,Marketed
8978,Darvocet-N 50 and...,propoxyphene naps...,2,887,null,null,pharma/bio,new combination,1454,12790,Marketed,null,9,1382,null,null,null,5,null,null,null,null,null,null,1465,null,null,null,product owner,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1972-12-19,null,null,not active,2010-11-19,Adverse Reactions...,8978,2017-10-04 15:05:27,2006-03-23 17:32:36,Marketed
8980,Darvon-N Tablets,Darvon N100,2,null,null,null,pharma/bio,new formulation,1454,12792,Marketed,relief of mild to...,9,1382,null,null,null,5,null,null,null,null,null,null,1465,null,null,null,product owner,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1971-09-09,null,1971-07-19,not active,2010-11-19,"How Supplied,Indi...",8980,2017-10-04 15:05:27,2006-03-23 17:32:36,Marketed
30285,Darvon-N Tablets 2,null,2,null,null,null,pharma/bio,new formulation,1454,12792,Marketed,Mild to moderate ...,9,1382,null,null,null,5,null,null,null,null,null,null,1465,626,null,null,product owner,null,null,null,null,27,null,null,null,null,null,null,null,null,null,null,null,null,null,8980,1973-12-31,null,null,not active,null,"How Supplied,Indi...",8980,2017-10-04 15:05:27,2009-08-22 11:17:25,Marketed
115108,Darvocet Tablets,null,2,887,null,null,pharma/bio,new combination,1454,null,Marketed,null,9,1382,null,null,null,null,null,null,null,null,null,null,5,null,null,null,product owner,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1972-10-18,null,null,not active,2014-02-21,"Company,Indicatio...",115108,2017-10-04 15:05:27,2014-02-19 17:15:35,Marketed
209273,Darvocet-N 50 and...,null,2,887,null,null,pharma/bio,generic,1454,null,Approved,null,9,1382,null,null,null,null,null,null,null,null,null,null,680,null,null,null,product o

In [53]:
print(list(compound.schema))

[StructField(compound_id,StringType,true), StructField(product_name,StringType,true), StructField(product_synonym,StringType,true), StructField(molecule_id_1,StringType,true), StructField(molecule_id_2,StringType,true), StructField(molecule_id_3,StringType,true), StructField(molecule_id_4,StringType,true), StructField(type,StringType,true), StructField(generic_or_innovator,StringType,true), StructField(therapeutic_category_id,StringType,true), StructField(therapeutic_group_id,StringType,true), StructField(phase,StringType,true), StructField(indication_explanation,StringType,true), StructField(route_branch_id,StringType,true), StructField(drug_delivery_branch_id_1,StringType,true), StructField(drug_delivery_branch_id_2,StringType,true), StructField(drug_delivery_branch_id_3,StringType,true), StructField(drug_delivery_branch_id_4,StringType,true), StructField(product_originator_company_id_1,StringType,true), StructField(product_originator_company_id_2,StringType,true), StructField(dds_te

In [ ]:
compound_comb = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_compound_combined.csv")

In [ ]:
compound_comb

### Molecule

In [147]:
molecule = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_molecule.csv")

In [148]:
# has molecules in pipelines
molecule

molecule_id,molecule_name,synonym,unii_number,CAS_registry
2,propoxyphene naps...,(+)- Propoxyphene...,null,17140-78-2
887,paracetamol,acetaminophen||ac...,UNII-362O9ITL9D,103-90-2
3,morphine sulfate ...,Di(morphine) dihy...,UNII-X3P646A2J0,6211-15-0
344,clindamycin phosp...,clindamycini phos...,UNII-EH6D7113I8,24729-96-2
2888,glycerin,glycerol||propane...,null,56-81-5
3715,citric acid,4-03-00-01272 ||A...,UNII-XF417D3PSL,77-92-9
3751,benzyl alcohol,BnOH||(Hydroxymet...,null,100-51-6
24631,potassium hydroxide,Caswell No. 693||...,UNII-WZH3C48M4T,1310-58-3
6,amlexanox,AA673||AA-673||AA...,UNII-BRL1C2459K,68302-57-8
1256,estradiol hemihyd...,beta-Estradiol se...,UNII-CXY7B3Q98Z,35380-71-3


In [149]:
molecule.agg(*(100.0*(F.count(F.col(c))/molecule.count()).alias(c) for c in molecule.columns))

((count(molecule_id) / 3984) AS `molecule_id` * 100.0),((count(molecule_name) / 3984) AS `molecule_name` * 100.0),((count(synonym) / 3984) AS `synonym` * 100.0),((count(unii_number) / 3984) AS `unii_number` * 100.0),((count(CAS_registry) / 3984) AS `CAS_registry` * 100.0)
100.0,100.0,81.70180722891565,53.26305220883534,90.06024096385542


In [146]:
territory = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_territory.csv")

In [ ]:
# territory seems goes withs ales
territory

In [ ]:
sales_comp = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/Sales_Compound.csv")

In [ ]:
sales_comp

In [ ]:
sales = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/Sales.csv")

In [ ]:
sales

In [ ]:
pc_form = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv("/home/zhengrongma/supply_chain_project/data/pharmacircle/pc_form.csv")

In [ ]:
pc_form

In [ ]:
path = "/home/zhengrongma/supply_chain_project/data/pharmacircle/"

file = "pc_form.csv"



test = pc_form = spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ";").csv(path + file)


In [ ]:
test.count()

In [55]:
gsrs =  spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("delimiter", ",").csv("/home/zhengrongma/supply_chain_project/data/unii_zip/gsrs.csv")

In [56]:
gsrs

unii,name,detail,type
486026U8ME,HUMAN ADENOVIRUS ...,structurallyDiverse,self
486026U8ME,HUMAN ADENOVIRUS ...,cn,name
486026U8ME,MASTADENOVIRUS: H...,sys,name
486026U8ME,HUMAN ADENOVIRUS D9,cn,name
486026U8ME,MASTADENOVIRUS: H...,sys,name
486026U8ME,10527,NCBI TAXONOMY,code
A1HKS2R89E,PODOCARPUS NERIIF...,structurallyDiverse,self
A1HKS2R89E,PODOCARPUS ENDLIC...,sys,name
A1HKS2R89E,NAGEIA LEPTOSTACH...,cn,name
A1HKS2R89E,NAGEIA DISCOLOR W...,cn,name


In [119]:
name_lookup = ( gsrs
                .filter(gsrs.type == 'self')
                .join( gsrs.withColumnRenamed('name','synonym'),'unii')
               .drop('detail', 'type')
              )  

In [129]:
name_lookup = name_lookup.withColumn('synonym_formated', F.regexp_replace(F.col('synonym'), r'\[(.*?)\]', ''))


In [138]:
name_lookup = name_lookup.withColumn('synonym_formated', F.trim(F.lower(F.col('synonym_formated'))))

In [142]:
matched = api_manufacturer.select(F.lower(F.col('api')).alias('api_lower')).join(name_lookup,
                                                           F.col('api_lower') == F.lower(F.col('synonym_formated')) ).select('api_lower','unii','synonym_formated').dropDuplicates()

In [143]:
#first pass match 9409 
# convert to lower case and strip [] 9414
matched.count()

9349

In [144]:
matched

api_lower,unii,synonym_formated
atropine sulfate,03J5ZE7KA5,atropine sulfate
atropine sulphate,03J5ZE7KA5,atropine sulphate
atropine sulfate ...,03J5ZE7KA5,atropine sulfate ...
thiopropazate,0JFY081Q2X,thiopropazate
laurocapram,1F3X9DRV9X,laurocapram
azone,1F3X9DRV9X,azone
7-keto-dehydroepi...,2334LJD2E9,7-keto-dehydroepi...
metopimazine,238S75V9AV,metopimazine
etofibrate,23TF67G79M,etofibrate
sodium carbonate ...,2A1Q1Q3557,sodium carbonate ...


In [93]:
gsrs.show(40, False)

+----------+-------------------------------------------------------+-------------------+----+
|unii      |name                                                   |detail             |type|
+----------+-------------------------------------------------------+-------------------+----+
|486026U8ME|HUMAN ADENOVIRUS D SEROTYPE 9                          |structurallyDiverse|self|
|486026U8ME|HUMAN ADENOVIRUS D SEROTYPE 9                          |cn                 |name|
|486026U8ME|MASTADENOVIRUS: HUMAN MASTADENOVIRUS D SEROTYPE 9 WHOLE|sys                |name|
|486026U8ME|HUMAN ADENOVIRUS D9                                    |cn                 |name|
|486026U8ME|MASTADENOVIRUS: HUMAN MASTADENOVIRUS D SEROTYPE 9      |sys                |name|
|486026U8ME|10527                                                  |NCBI TAXONOMY      |code|
|A1HKS2R89E|PODOCARPUS NERIIFOLIUS WHOLE                           |structurallyDiverse|self|
|A1HKS2R89E|PODOCARPUS ENDLICHERIANUS WHOLE                 

In [100]:
import re
str = "PALMITAMIDOPROPYL DIMETHYLAMINE [INCI]"
print(re.sub(r'\[(.*?)\]', '', str).strip())

PALMITAMIDOPROPYL DIMETHYLAMINE


In [118]:
name_lookup.withColumn('synonym_formated', F.regexp_replace(F.col('synonym'), r'\[(.*?)\]', ''))

unii,name,synonym,synonym_formated
01343Q8EL8,ANISODAMINE,ANISODAMINE,ANISODAMINE
01343Q8EL8,ANISODAMINE,ANISODAMINE,ANISODAMINE
01343Q8EL8,ANISODAMINE,(-)-6.BETA.-HYDRO...,(-)-6.BETA.-HYDRO...
01343Q8EL8,ANISODAMINE,"HYOSCYAMINE, 6-HY...","HYOSCYAMINE, 6-HY..."
01343Q8EL8,ANISODAMINE,BENZENEACETIC ACI...,BENZENEACETIC ACI...
01343Q8EL8,ANISODAMINE,ANISODAMINE [WHO-DD],ANISODAMINE
01343Q8EL8,ANISODAMINE,ANISODAMINE [MART.],ANISODAMINE
01343Q8EL8,ANISODAMINE,6.BETA.-HYDROXYHY...,6.BETA.-HYDROXYHY...
01343Q8EL8,ANISODAMINE,SUB25941,SUB25941
01343Q8EL8,ANISODAMINE,ANISODAMINE,ANISODAMINE


Join between molecule and api

In [154]:
molecule_api = api_manufacturer.select(F.lower(F.col('api')).alias('api_lower')).join(molecule,
                                                                                       F.col('api_lower') == F.col('molecule_name')
                                                                                     ).dropDuplicates()
              

In [155]:
molecule_api.count()

2718